# 2.1 理解word embeddings

1.LLM 无法直接处理raw text，因为文本是分类数据，与用于实现神经网络的数学运算是不兼容的。因此我们就需要一种将原始数据转换为向量形式的工具。<br>

2.embedding的概念：将数据（可以是多种格式：text, video, audio）转换为向量(vector)格式。embedding实际上就是将离散的对象转换为连续向量空间中的点的过程，embedding的目的就是将非数值（non-numeric）的数据转换为神经网络可以处理的格式。<br>

3.不同的数据格式，需要的embedding模型是不一样的，比如文本的和视频的就不一样。并且，同样是对于文本嵌入（text embedding），也会存在多种嵌入的类型，比如word embedding, sentence/paragraphs/whole documents embedding等。Sentence/paragraphs embedding 是检索增强生成（retrieval-augmented generation , RAG）比较常用的embedding格式。<br>

Note：<br>
RAG是将生成和检索（搜索外部知识库）结合，用于拉取相关知识然后生成文本的方法。<br>

4.最常用的word embedding工具是Word2Vec方法，实际上就是用预训练好的神经网络来预测给定目标词的上下文或者给定上下文来预测目标词，来生成词嵌入。<br>
在word embedding中，相似概念的词，在词嵌入空间中的距离就离得越近。<br>

5.Word embedding的维度可以从1到上千那么大，但是维度越高一方面可以捕捉更加细微的关系，但是计算资源的消耗也会更大。Embedding size一般指的是模型的隐藏状态(hidden states)的维度。<br>

6.LLM一般不会使用传统的词嵌入模型来生成word embedding，而是有专门的embedding layer，并且是trainable的，主要就是为了能够在具体的任务和数据上进行优化和适配。<br>

# 2.2 Tokenizeing text

1.Token指的是独立的单词或者特殊的字符，也包括标点符号。<br>

2.在Tokenization过程中，通常要避免将text中的所有字母都变成小写了，因为大小写通常会帮助LLM区分专有名词和普通名词、理解句子结构、并且学习如何在输出中生成正确的大小写。<br>

3.在训练tokenizer的时候，是否要将空格也编码为独立的符号，需要根据实际的应用场景来判断。比如在我们对文本的结构十分敏感的时候，就需要保留空格，比如在代码生成任务中的时候。<br>

# 2.3 将Token转换为Token IDs

1.将从text进行tokenization之后得到的tokens转换为数值表示，以生成所谓的 token IDs。这是将token IDs转换为embedding vectors的前序步骤。<br>

2.为了实现这个转换，我们必须构建一个 vocabulary，它定义了如何建每一个独特的单词和符号转换为独特的数值。<br>

3.在构建这个vocabulary之前，通常需要将这些tokens按照字母表的顺序首字母排列，并且去重，然后将这些独特的token映射为独特的数值。<br>

4.上面3讲的这个过程是从token ——》 token IDs，这是LLM输入阶段需要进行的处理。如果是LLM输出的token IDs，我们应该建立一个反向的映射vocabulary，将LLM的输出映射回字符串。<br>

5.一个完整的tokenizer类必须包含两个方法：<br>
（1）encode:将文本划分为tokens，并且实施string-to-integer的映射，通过vocabulary来产生token IDs。<br>
（2）decode：实施integer-to-string的映射，将token IDs转换回文本text。<br>

# 2.4 添加Context Tokens

1.Context Tokens：<br>
（1）<|unk|>：表示新的或者未知的单词，实际上就是训练数据中没有的，并且在字典中没有记录的单词；遇到字典中没有的单词的时候使用。<br>

（2）<|endoftext|>:用于分隔两个不相关的文本，标志着一段文本的结束或者开始；<br>

（3）[BOS](begining of sequence): 标志着一段文本的开始；<br>

（4）[EOS](end of sequence)：放置在文本的末尾，主要用于连接多个不相关的文本的，与<|endoftext|>作用相同；<br>

（5）[PAD](padding)：一般是在LLM的batch sizes大于一的时候，一个batch中有不同长度的text，为了使得所有text的长度一致，就会以最长的text的长度为标准，把长度小于此的text扩展成一样长，而扩张就是使用的[PAD]这个token。<br>
在LLM中，用于扩展（padding）操作的token除了[PAD]以外，也会使用<|endoftext|>，但是实际上LLM会使用一个mask，来掩蔽那些新增的padded token，因此使用什么token来进行padding不重要，因为这些padded token都会被忽略掉。<br>

2.注意：<br>
（1）GPT这些模型除了使用<|endoftext|> token之外，不会使用以上的其他任何token。<br>
（2）GPT使用的tokenizer对于那些不在字典中的单词，也不会使用<|unk|> token，而是使用byte pair encoding (BPE) tokenizer，主要就是将单词(words)划分为子词单元(subword units)。<br>